# Fear and Greed Index scraped from CNN

In [1]:
import pandas as pd
import requests, csv, yfinance, pytz, json
from datetime import datetime
import datetime
import os.path
import re
import tempfile
import typing
import pytz
import requests


## scrapping from CNN

In [2]:
!pip install yfinance
!pip install backtrader[plotting]
import requests, csv, yfinance, pytz, json
from datetime import datetime
base_url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
start_date = "2020-09-18"
r= requests.get("{}/{}".format(base_url, start_date))
data = r.json()
print(json.dumps(data, indent = 2))

In [3]:
fg_data = data['fear_and_greed_historical']['data']

fear_greed_values = {}

FEAR_GREED_CSV_FILENAME = 'fear-greed.csv'

with open(FEAR_GREED_CSV_FILENAME, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Date,fear_greed'])

    for data in fg_data:
        dt = datetime.fromtimestamp(data['x'] / 1000, tz=pytz.utc)
        fear_greed_values[dt.date()] = int(data['y'])
        writer.writerow([dt.date(), "", "", "", int(data['y'])])


In [4]:
# pre-downloaded csv
df = pd.read_csv("fear_greed2020.csv", index_col = "date")

In [5]:
df.head()

,fear_greed_index
date,
2020-09-18,50
2020-09-21,50
2020-09-22,50
2020-09-23,50
2020-09-24,50


In [6]:
# cleaning the data
df.index= df.index.str.strip().str.lower()


In [7]:
df.head()

,fear_greed_index
date,
2020-09-18,50
2020-09-21,50
2020-09-22,50
2020-09-23,50
2020-09-24,50


In [8]:
df2 = pd.read_csv("fear-greed.csv", index_col = "date")

In [9]:
df2.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,fear_greed_index_1
date,,,,
2011-01-03,NaN,NaN,NaN,68
2011-01-04,NaN,NaN,NaN,68
2011-01-05,NaN,NaN,NaN,67
2011-01-06,NaN,NaN,NaN,64
2011-01-07,NaN,NaN,NaN,63


In [10]:
#cleaning the columns
df2 = df2.dropna(axis=1, how='all')

In [11]:
df2.head()

,fear_greed_index_1
date,
2011-01-03,68
2011-01-04,68
2011-01-05,67
2011-01-06,64
2011-01-07,63


In [12]:
# there is one duplicate date on both datafremes
# pd.merge(df, df2, left_index=True, right_index=True)


In [13]:
# duplicate index
df = df.drop("2020-09-18")

In [14]:
df.head()

,fear_greed_index
date,
2020-09-21,50
2020-09-22,50
2020-09-23,50
2020-09-24,50
2020-09-25,50


In [21]:
# combining the two csv's
fear_greed = df2.append(df)
fear_greed.head()

,fear_greed_index_1,fear_greed_index
date,,
2011-01-03,68.0,NaN
2011-01-04,68.0,NaN
2011-01-05,67.0,NaN
2011-01-06,64.0,NaN
2011-01-07,63.0,NaN


In [16]:
fear_greed.columns

Index(['fear_greed_index_1', 'fear_greed_index'], dtype='object')

In [23]:
# combining both columns and removing the nulls
fear_greed['fear_greed_index'].update(fear_greed.pop('fear_greed_index_1'))


In [24]:
fear_greed.isna().mean()

fear_greed_index    0.0
dtype: float64

In [25]:
fear_greed.head()

,fear_greed_index
date,
2011-01-03,68.0
2011-01-04,68.0
2011-01-05,67.0
2011-01-06,64.0
2011-01-07,63.0


In [26]:
fear_greed.tail()

,fear_greed_index
date,
2022-10-27,57.0
2022-10-28,60.0
2022-10-31,58.0
2022-11-01,57.0
2022-11-01,57.0


In [27]:
fear_greed.to_csv("fear_greed_index", index = "date")